In [1]:
# Packages
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import random
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from sklearn.impute import SimpleImputer

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 2: Classification with SVM, BP and MLR

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez

Part 1: Selecting and analyzing the datasets

We will perform the classification in the following three datasets:

In [9]:
# We upload the datasets

# First dataset: File: A2-ring.txt
    # Training set 1 : ring-separable.txt
    # Training set 2 : ring-merged.txt
    # Two different training sets, one easy (separable) and one more difficult (merged)

    # Test (valid for set1 and set2): ring-test.txt (Only one test set for both training sets)
    # 2 input features + 1 class identifier (0 / 1)
    # All data files have 10000 patterns
    
A2_ring_merged=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-ring/A2-ring-merged.txt', sep='\t', header=None)
A2_ring_separable=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-ring/A2-ring-separable.txt', sep='\t', header=None)
A2_ring_test=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-ring/A2-ring-test.txt', sep='\t', header=None)

df_A2_ring_merged=pd.DataFrame(A2_ring_merged)
df_A2_ring_separable=pd.DataFrame(A2_ring_separable)
df_A2_ring_test=pd.DataFrame(A2_ring_test)

# We plot the two input features

In [19]:
# Second dataset: File: A2-bank.txt
    # Data: bank-additional.csv (4119 patterns) or bank-additional-full.csv (41188 patterns), we choose one of them (the first is a subset of the second)
    # Training: select the first 80% patterns for training
    # Test: select the last 20% patterns for test
    # Features: 20 features, most of them categorical, you will have to properly represent them as numerical data before training
    # Input features: features that refer to the bank client, last contact in the current campaign, other attributes, and social and economic context attributes
    # Prediction feature: the last one (yes/no), which corresponds to whether the client has subscribed a term deposit or not
    # Observation: missing information is tagged as “unknown”

bank_additional=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-bank/bank-additional.csv', sep=';', header=None)
#bank_additional_full=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-bank/bank-additional-full.csv', sep=';', header=None)
#bank_additional_names= pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-bank/bank-additional-names.txt', sep="\t", header=None)

df_bank_additional=pd.DataFrame(bank_additional)
#df_bank_additional_full=pd.DataFrame(bank_additional_full)

In [29]:
# Third dataset: from "https://www.kaggle.com/datasets/fatemehmehrparvar/liver-disorders?resource=download"
    # At least 6 features, one of them used for classification
    # he classification feature can be binary or multivariate
    # At least 400 patterns
    # Select randomly 80% of the patterns for training and validation, and the remaining 20% for test; it is important to shuffle the original data, to destroy any kind of sorting it could have

# Indian liver patient dataset [584 rows x 11 columns]
liver_Disorder=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/Indian Liver Patient Dataset (ILPD).csv', sep=',', header=None)

# data (as pandas dataframes) 
df_liver_Disorder=pd.DataFrame(liver_Disorder)

header_vector_liver_Disorder = df_liver_Disorder.columns.tolist() #header

In [31]:
print(header_vector_liver_Disorder)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [33]:
print(df_liver_Disorder)

      0       1     2    3        4     5     6    7    8          9   \
0    Age  Gender    TB   DB  Alkphos  Sgpt  Sgot   TP  ALB  A/G Ratio   
1     65  Female   0.7  0.1      187    16    18  6.8  3.3        0.9   
2     62    Male  10.9  5.5      699    64   100  7.5  3.2       0.74   
3     62    Male   7.3  4.1      490    60    68    7  3.3       0.89   
4     58    Male     1  0.4      182    14    20  6.8  3.4          1   
..   ...     ...   ...  ...      ...   ...   ...  ...  ...        ...   
579   60    Male   0.5  0.1      500    20    34  5.9  1.6       0.37   
580   40    Male   0.6  0.1       98    35    31    6  3.2        1.1   
581   52    Male   0.8  0.2      245    48    49  6.4  3.2          1   
582   31    Male   1.3  0.5      184    29    32  6.8  3.4          1   
583   38    Male     1  0.3      216    21    24  7.3  4.4        1.5   

           10  
0    Selector  
1           1  
2           1  
3           1  
4           1  
..        ...  
579        